# FROM SCRATCH
## custom numpy autograd
This notebook is intended for the submission to test the code.

The results were achieved running train_wandb.py and train_wandb_2nd_dataset.py

Dataset are reduced for faster run speed 

In [ ]:
from importlib.resources import path
import numpy as np
import engine
import nn
import optim
import transforms

class MockWandB:
    def __init__(self, config):
        self.config = type('Config', (), config)
    def log(self, data):
        pass
    def finish(self):
        pass

def download_dataset():
    with np.load('cifar_dataset.npz') as data:
        x_train = data['x_train']
        y_train = data['y_train']
        x_test = data['x_test']
        y_test = data['y_test']
    return x_train, y_train, x_test, y_test

def downscale_smart_numpy(images):
    n, h, w, c = images.shape
    reshaped = images.reshape(n, h // 2, 2, w // 2, 2, c)
    downscaled = reshaped.mean(axis=(2, 4))
    return downscaled

def load_data():
    try:
        x_train, y_train, x_test, y_test = download_dataset()
    except FileNotFoundError:
        print("Dataset file not found")
        return None, None, None, None
    
    y_test_raw = y_test.flatten().astype(int)
    y_train_raw = y_train.flatten().astype(int)

    x_train = x_train.astype(np.float32) / 255.0
    x_test = x_test.astype(np.float32) / 255.0
    
    mean = np.mean(x_train, axis=(0,1,2), keepdims=True)
    std = np.std(x_train, axis=(0,1,2), keepdims=True)
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std
    
    x_train = x_train[:100]
    y_train_raw = y_train_raw[:100]
    x_test = x_test[:50]
    y_test_raw = y_test_raw[:50]

    classes = 10
    y_train = np.eye(classes, dtype=np.float32)[y_train_raw]
    y_test = np.eye(classes, dtype=np.float32)[y_test_raw]
    
    x_train = x_train.reshape(x_train.shape[0], -1) 
    x_test = x_test.reshape(x_test.shape[0], -1)    
    
    return x_train, y_train, x_test, y_test

def evaluate(model, x_test, y_test):
    model.eval()
    with engine.no_grad():
        inputs = engine.Tensor(x_test)
        targets = engine.Tensor(y_test)
        preds = model(inputs)
        pred_labels = np.argmax(preds.data, axis=1)
        true_labels = np.argmax(targets.data, axis=1)
        acc = np.mean(pred_labels == true_labels)
    return acc


In [ ]:
def main():
    config = {
        "learning_rate": 0.001,
        "epochs": 2,
        "batch_size": 16,
        "architecture": "MLP-Reduced",
        "dataset": "CIFAR10",
        "optimizer": "AdamW",          
        "momentum": 0.9,                
        "weight_decay": 1e-4,           
        "augmentation": "RandomRotation(+/- 15)"
    }
    
    wandb = MockWandB(config)
    
    lr = wandb.config.learning_rate
    batch_size = wandb.config.batch_size
    epochs = wandb.config.epochs
    weight_decay = wandb.config.weight_decay
    optim_name = wandb.config.optimizer
    momentum = wandb.config.momentum
    
    x_train, y_train, x_test, y_test = load_data()
    if x_train is None: return

    num_samples = x_train.shape[0]

    model = nn.Sequential(
        nn.Linear(3072, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 10)
    )

    criterion = nn.CrossEntropyLoss()

    if optim_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optim_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    
    train_transform = transforms.Compose([
        transforms.RandomRotation(degrees=15, image_size=32, channels=3)
    ])

    for epoch in range(epochs):
        model.train() 
        epoch_loss = 0
        if epoch % 5 == 0:
            optimizer.lr -= 0.00005  
        
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        x_train = x_train[indices]
        y_train = y_train[indices]

        for start in range(0, num_samples, batch_size):
            end = start + batch_size
            x_batch_np = x_train[start:end]
            y_batch = engine.Tensor(y_train[start:end])

            x_batch_aug = train_transform(x_batch_np)
            x_batch = engine.Tensor(x_batch_aug)

            preds = model(x_batch)
            loss = criterion(preds, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += float(loss.data)

        avg_train_loss = epoch_loss / (num_samples // batch_size)
        test_acc = evaluate(model, x_test, y_test)
        
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Test Acc: {test_acc*100:.2f}%")
        
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "test_accuracy": test_acc
        })

    wandb.finish()

if __name__ == "__main__":
    main()
